In [12]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
from data_normalizer import unscale_data_with_minmax
from sklearn.metrics import make_scorer
import joblib

In [15]:

filtered_data = np.load('./macroeconomic_data/filtered_data.npy', allow_pickle=True).item()

data = []

for date, data_frame in filtered_data.items():
    flattened_data = pd.json_normalize(data_frame, sep=".")
    flattened_data["10"] = str(date[1])
    data.append(flattened_data)

data = pd.concat(data, ignore_index=True)

X = data
X = X.iloc[1:, :]

y = data.filter(regex='0.36', axis=1)  # '^0\.'
y = y.iloc[:-1]
y = y.values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [7]:
base_estimator = MLPRegressor()
parameters = {'hidden_layer_sizes': [(200, 200, 200, 200, 200), (200,200,200,200), (100, 200, 200, 100), (200,200,200)],
              "activation": ['logistic', 'tanh'],
              "solver": ['sgd'], "alpha": [0.001, 0.0001, 0.00005],
              "learning_rate": ['invscaling', 'adaptive'],
              'max_iter': [100000], #'early_stopping': [True],
              'tol': [1e-6], 'momentum' : [0.8, 0.85, 0.9, 0.95], 'nesterovs_momentum': [True, False]
              }


In [8]:

mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

clf = GridSearchCV(base_estimator, param_grid=parameters, cv=5, error_score='raise', n_jobs=-1, verbose=2, scoring=mse_scorer)
clf.fit(X_train, y_train)

best_estimator = clf.best_estimator_

joblib.dump(best_estimator, './joblib/best_estimator.joblib', 3)


Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[CV] END activation=logistic, alpha=0.001, hidden_layer_sizes=(200, 200, 200, 200, 200), learning_rate=invscaling, max_iter=100000, momentum=0.8, nesterovs_momentum=True, solver=sgd, tol=1e-06; total time=  15.2s
[CV] END activation=logistic, alpha=0.001, hidden_layer_sizes=(200, 200, 200, 200, 200), learning_rate=invscaling, max_iter=100000, momentum=0.85, nesterovs_momentum=False, solver=sgd, tol=1e-06; total time=   7.1s
[CV] END activation=logistic, alpha=0.001, hidden_layer_sizes=(200, 200, 200, 200, 200), learning_rate=invscaling, max_iter=100000, momentum=0.9, nesterovs_momentum=True, solver=sgd, tol=1e-06; total time=   6.1s
[CV] END activation=logistic, alpha=0.001, hidden_layer_sizes=(200, 200, 200, 200, 200), learning_rate=invscaling, max_iter=100000, momentum=0.9, nesterovs_momentum=False, solver=sgd, tol=1e-06; total time=   2.9s
[CV] END activation=logistic, alpha=0.001, hidden_layer_sizes=(200, 200, 200, 200

In [ ]:
print(clf.cv_results_)

In [ ]:
try:
    best_estimator
except:
    best_estimator = joblib.load('./joblib/best_estimator.joblib')

In [ ]:
print(best_estimator)

In [ ]:
y_pred = best_estimator.predict(X_test)

y_pred_scaled = unscale_data_with_minmax(np.array(y_pred))
y_test_scaled = unscale_data_with_minmax(np.array(y_test))

# Print the predicted and actual values side by side
print("Predicted\tActual")
for pred, actual in zip(y_pred_scaled, y_test_scaled):
    print(f"{pred}\t\t{actual}")

mse = mean_squared_error(y_test_scaled, y_pred_scaled)
r2 = r2_score(y_test_scaled, y_pred_scaled)
print('Mean Squared Error:', mse)
print('R-squared Score:', r2)


In [ ]:
plt.scatter(y_pred_scaled, y_test_scaled)
plt.plot([99, 101], [99, 101])
plt.title("Test data")
plt.show()

y_train_scaled = unscale_data_with_minmax(y_train)
y_train_pred = unscale_data_with_minmax(clf.predict(X_train))
plt.scatter(y_train_pred, y_train_scaled)
plt.plot([99, 101], [99, 101])
plt.title("Train data")
plt.show()

plt.figure()
plt.plot(best_estimator.loss_curve_)
plt.xlabel("iteration_no")
plt.ylabel("loss")
plt.show()

plt.figure()
plt.plot(best_estimator.validation_scores_)
plt.xlabel("iteration_no")
plt.ylabel("validation")
plt.show()
